In [4]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

In [5]:
load_dotenv()
google_api_key=os.getenv("google_api_key")

if not google_api_key:
    raise ValueError("API Key not found in .env file!")

model=ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=google_api_key)

In [6]:
model.invoke("HI")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--31e4aa2c-778b-4fda-808f-7666603ed1e2-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}})

In [7]:
class LLMState(TypedDict):
    question: str
    answer: str

In [8]:
def llm_qa(state: LLMState)->LLMState:
    question=state['question']
    prompt=f'Answer the following question {question}'
    answer=model.invoke(prompt).content

    state['answer']=answer

    return state

In [9]:
graph=StateGraph(LLMState)

graph.add_node('llm_qa',llm_qa)

graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

workflow=graph.compile()

In [15]:
initial_state={'question':'How far is moon from earth'}
final_state=workflow.invoke(initial_state)
print(final_state['answer'])

The average distance from the Earth to the Moon is approximately **384,400 kilometers (238,900 miles)**.

However, this distance isn't constant. The Moon's orbit around Earth is elliptical, meaning the distance varies throughout its orbit:

*   **Perigee (closest point):** Around 363,104 km (225,623 miles)
*   **Apogee (farthest point):** Around 405,696 km (252,088 miles)

It takes light about 1.28 seconds to travel from the Moon to the Earth.


In [16]:
model.invoke('how far is moon from the earth?').content

"The distance between the Earth and the Moon is not constant because the Moon's orbit around Earth is elliptical, not a perfect circle.\n\nHere are the key figures:\n\n*   **Average Distance:** Approximately **384,400 kilometers (km)** or **238,900 miles**.\n*   **Closest Point (Perigee):** Around **363,104 km (225,623 miles)**.\n*   **Farthest Point (Apogee):** Around **405,696 km (252,088 miles)**.\n\nTo put that into perspective, you could fit all the other planets in our solar system (Mercury, Venus, Mars, Jupiter, Saturn, Uranus, Neptune) side-by-side between the Earth and the Moon during their average distance!"